In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import AdamW

In [2]:
batch_size = 32
block_size = 128
dim_embd = 384
num_heads = 6
n_blocks = 4
dropout=0.2
head_size = 384
learning_rate = 3e-4
ffrwd_in_features = 384

max_iters = 2500
eval_interval = 500
eval_iters = 100


device = 'cuda' if torch.cuda.is_available() else 'cpu'
seed = 1337
g = torch.manual_seed(seed)


In [3]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [4]:
with open('./input.txt') as f:
    text = f.read()
print(f"{len(text) = } characters", end='\n-------------Data-------------\n')
print(text[:100])

len(text) = 1115394 characters
-------------Data-------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### Build our char level vocabulary

In [5]:
chars = sorted(list(set(text)))
stoi = {ch:idx for idx, ch in enumerate(chars)}
itos = {idx:ch for idx, ch in enumerate(chars)}
vocab_size = len(chars)
print(vocab_size)

65


In [ ]:
torch.save(stoi, './stoi.pt')

In [6]:
encode = lambda text: [stoi[c] for c in text]
decode = lambda tokens: ''.join([itos[c] for c in tokens])
raw = 'i love this game.'
tokens = encode(raw)
print(tokens)
print(decode(tokens))

[47, 1, 50, 53, 60, 43, 1, 58, 46, 47, 57, 1, 45, 39, 51, 43, 8]
i love this game.


### Encode entire data

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type())
print(data[:100])

torch.Size([1115394]) torch.LongTensor
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### Data split

In [8]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(train_data[:block_size])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1])


In [9]:
def get_batch(split_type):
    data = train_data if split_type=='train' else val_data
    ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,), generator=g)
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y
xb, yb = get_batch('train')
print(xb.shape, yb.shape)

torch.Size([32, 128]) torch.Size([32, 128])


In [10]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Mathematical Trick to vectorized weighted aggregation

In [11]:
B,T,C = (4,8,2)
x = torch.randn(B,T,C)

In [12]:
## Version 1
xbow = torch.zeros_like(x)
for t in range(T):
    x_prev = x[:, :t+1, :]
    xbow[:,[t], :] = torch.mean(x_prev, dim=1, keepdim=True)

In [13]:
## Version 2
wei = torch.tril(torch.ones((T,T))).float()
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow2, xbow)

True

In [14]:
## Version 3: using softmax
tril = torch.tril(torch.ones((T,T))).float()
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

### Decoder only transformer

In [15]:
class SelfAttention_Head(nn.Module):
    """One head of self-attention"""
    def __init__(self, head_features):
        super().__init__()
        self.head_features = head_features

        self.WQ = nn.Linear(dim_embd, head_features, bias=False)
        self.WK = nn.Linear(dim_embd, head_features, bias=False)
        self.WV = nn.Linear(dim_embd, head_features, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        B,T,C = x.shape
        q = self.WQ(x)
        k = self.WK(x)

        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        ## this mask to disconnect the token from following tokens in the seqeunce
        ## we will use this mask during training only
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        attention_weights = F.softmax(wei, dim=-1) ## this is attention weights
        attention_weights = self.dropout(attention_weights)

        v = self.WV(x)
        out = attention_weights @ v
        return out

In [16]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_features):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention_Head(head_features) for _ in range(num_heads)])
        ## Projection to make output compatible with residual adding
        self.proj = nn.Linear(head_features*num_heads, dim_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        x = self.dropout(x)
        return x

In [17]:
class FeedForward(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features, in_features * 4),
            nn.ReLU(),
            ## Projection to make output compatible with residual adding
            nn.Linear(in_features * 4, in_features),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.fc(x)

In [18]:
## LayerNorm: same as BatchNorm ,but it normalize the rows not the columns
class Block(nn.Module):
    def __init__(self, input_features, num_heads):
        super().__init__()
        head_size = input_features // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(input_features)
        self.ln1 = nn.LayerNorm(input_features)
        self.ln2 = nn.LayerNorm(input_features)

    def forward(self, x):
        ## (x +) is for residual connections
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [19]:
class Attentioned_LM(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_size = block_size
        self.embd_layers = nn.Embedding(vocab_size, dim_embd)
        self.position_encoding_layer = nn.Embedding(block_size, dim_embd)
        self.blocks = nn.Sequential(*[Block(dim_embd, num_heads=head_size) for _ in range(n_blocks)])
        self.ln_f = nn.LayerNorm(dim_embd)
        self.lm_head = nn.Linear(dim_embd, vocab_size)
        self.apply(self._init_weights)
    

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, idx, targets=None):
        B, T = idx.shape

        embdings = self.embd_layers(idx)
        pos_encodes = self.position_encoding_layer(torch.arange(T, device=device))
        x = embdings + pos_encodes ## (B,T,dim_embd) + (T,dim_embd) = (B,T,dim_embd) broadcast happened
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            batch, seq_len, embds = logits.shape
            logits = logits.view(batch*seq_len, embds)
            targets = targets.view(batch*seq_len)
            loss = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            # print(logits)
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            # print(probs)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

## In this code we still preserve history tokens, however we don't use them.


In [20]:
model = Attentioned_LM()
m = model.to(device)
optimizer = AdamW(m.parameters(), lr=learning_rate)

# input = torch.zeros((1, 1), dtype=torch.long).to(device)
# print(decode(m.generate(input, max_new_tokens=100)[0].tolist()))

In [21]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2022, val loss 4.2054
step 500: train loss 2.1278, val loss 2.1789
step 1000: train loss 1.7894, val loss 1.9363
step 1500: train loss 1.6039, val loss 1.7943
step 2000: train loss 1.5124, val loss 1.7204
step 2499: train loss 1.4440, val loss 1.6615


In [22]:
with torch.no_grad():
    m.eval()
    input = torch.zeros((1, 1), dtype=torch.long).to(device)
    print(decode(m.generate(input, max_new_tokens=1000)[0].tolist()))
    m.train()


HORD EOW:
Vophrouo! a procemed of design this head?
Shall he not good how night?

COMINIUS:
Ah, all as born,
I am hand!

POLIXENES:
I'll o'-make a I will let that ssort be, treal mine
Is that thou palenth, that shall hath contentl'd
For loss you let rother: if all bit by fine,
As how you love speak me, and by rags will grief
To contenter apperfure'd bang in as fair
Than your contry off his adver,
See, how a know holer, I'll and tall hear his sustil,
I the remaimsain come to his what the all how myself?
I never little us o' full all; I warrant
That ongle are-such die.

LUCIO:
Nay, if any for God's hand prit is king: thus, I had know
a not bleasted all ne'er bear deep do lin;
By be g, a droth Hencenge my busing please,
When must ill the flew your old stame of the mothers
Sorrow clourse of here; and a perser'd Husband.

CAPULET:
My lord, as by childer, go he would shall be
stay a all your repue's guard: so in horsest know's
good on grief no to mine sworn upon which you.

FRIAR LAURENCE:


In [ ]:
torch.save(m, './nanoGpt.bin')

In [24]:
# GG